In [8]:
import pandas as pd
# from tqdm import tqdm_notebook
import tqdm
import random
random.seed(42)
from pandas import ExcelWriter
import re
import urllib.request
import urllib.parse
import json

# Make HTML visualization of discussion, revisions before and after

## Build dataframe

In [3]:
# Load examples
# reject_arts = ['Priesthood keys', '33550336 (number)']
examples = pd.read_csv('/home/michael/school/research/wp/jsalt_tutorial/examples.csv', parse_dates=['timestamp'])
# examples = examples[examples['article'].isin([art for art in list(set(examples['article'])) if not art in reject_arts])]

In [4]:
pd.set_option('display.max_colwidth', -1)

In [22]:
# Attempt at Polish to German (name replacement)
examples.replace(to_replace={'article': {'Polish occupation of Czechoslovakia': 'German occupation of Czechoslovakia'}}, inplace=True)
examples.replace(to_replace={'article': {'Rubber tired metro': 'Rubber-tyred metro'}}, inplace=True)
examples

/home/michael/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


,contribution_id,article,thread,username,timestamp,thread_type,text,editor_score
0,6395,Acharya S,Delete actual name,RL0919,2013-11-20 04:52:07,THREAD_STARTER,"Acharya S responded on her Facebook page that due to her occupation she feels duress from stalkers knowing her ""actual name."" Per WP:BLPNAME: When the name of a private individual has not been widely disseminated or has been intentionally concealed, such as in certain court cases or occupations, it is often preferable to omit it, especially when doing so does not result in a significant loss of context. I ask that we remove the name. Raquel Baranow (talk) 16:29, 19 November 2013 (UTC)",0.706074
1,6397,Acharya S,Delete actual name,RL0919,2013-11-20 04:52:07,POST,":::To clarify, what she claims on Facebook is that the name given is not correct, but people who think it is her name use it in an attempt to intimidate her because they know she has been stalked in the past. The factual denial seems more relevant than the stalking concern. --RL0919 (talk) 10:52, 20 November 2013 (UTC)",0.706074
2,6396,Acharya S,Delete actual name,Capitalismojo,2013-11-19 22:24:03,POST,":Her ""stalkers"" must be unaware of this thing called ""google"". Capitalismojo (talk) 04:24, 20 November 2013 (UTC)\n::And books. Capitalismojo (talk) 04:26, 20 November 2013 (UTC)",0.416667
3,6412,Acharya S,Delete actual name,Capitalismojo,2013-12-26 18:46:15,POST,":::::::::::This has been resolved by the recent RfC (see above). If you would like to start another RfC again so soon hoping to get another result, feel free but know that this issue has been resolved. Capitalismojo (talk) 00:46, 27 December 2013 (UTC)",0.416667
4,6399,Acharya S,Delete actual name,TheRedPenOfDoom,2013-12-13 22:09:46,POST,"::someone who is out there publishing and promoting books is not a ""private individual."" -- TRPoD aka The Red Pen of Doom 04:09, 14 December 2013 (UTC)",0.659794
5,6410,Acharya S,Delete actual name,TheRedPenOfDoom,2013-12-28 11:25:03,POST,"::::::::::::::::Madonna Louise Ciccone is not her preference and appears in about the same proportion. Murdock is not a ""private"" individual, she is a public figure who has promoted her work. -- TRPoD aka The Red Pen of Doom 17:25, 28 December 2013 (UTC)",0.659794
6,6400,Acharya S,Delete actual name,In ictu oculi,2013-12-14 02:25:28,POST,":::Were the two editors directly above invited here by email? I think this was considered in the RfC and the 4 book sources all use the name ""Dorothy M. Murdock"" and two use ""Dorothy M___ Murdock"" while referencing it to the author's own website so this seems to be rather ridiculous. WP:FULLNAME applies to leads in all biographies including ""Chelsea Elizabeth Manning[4] (born Bradley Edward Manning, December 17, 1987) so I'm not sure what the comparison proves. If anything we shouldn't really have M. when Holding and the school source the middle name. In ictu oculi (talk) 08:25, 14 December 2013 (UTC)",0.309613
7,6401,Acharya S,Delete actual name,In ictu oculi,2013-12-25 18:57:04,POST,"::::TEMPLATE[ping, In ictu oculi] Sorry, I did not realize there had been an RfC. Full disclosure: I have interacted with the subject via OTRS. I changed to the initials because the subject has actually published under them unlike the full name. Are there instances of her having actually published under the name ""Dorothy"" or admitting that is here real name? Annual Report - American School of Classical Studies at Athens doesn't show any proof and the other two sources surely can't be said to be reliable for her full name. -- John Reaves 00:52, 26 December 2013 (UTC)\n::::::The two additional book print sources, one ""Dorothy M. Murdock"", on ""Dorothy Milne Murdock"" are reliable sources. The summer school Annual Report - American School of Classical Studies at Athens was introduced by the author herself on her own website as part of author's expertise/credentials. In ictu oculi (talk) 00:57, 26 December 2013 (UTC)",0.309613
8,6404,Acharya S,Delete ac

In [6]:
# Remove empty Rajsingam posts
examples = examples[examples['text'] != 'Rajsingam 26 Dec. 2006']

In [9]:
# For each thread
threads = list(set(zip(examples['article'], examples['thread'])))
r = random.random()
random.shuffle(threads, lambda: r)
threads

[('Acharya S', 'Delete actual name'),
 ('German occupation of Czechoslovakia',
  'Extent of coverage of the postwar expulsions'),
 ('Thermal light', 'Reworked'),
 ('Tour de France', 'Armstrong'),
 ('Satan (South Park)', 'Proposed removal of content'),
 ('Hangul', 'Requested move'),
 ('Mount Hebron', 'Removal of well sourced material'),
 ('Rubber tired metro',
  'Rubber Tired Metros as burdensome and having only political (not technical) merit'),
 ('Anton Balasingham', 'Asia Tribune vs. Tamilnet'),
 ('Croatia national football team', 'Pre independence or 1990s?')]

In [23]:
threads = [
 ('German occupation of Czechoslovakia',
  'Extent of coverage of the postwar expulsions'),
 ('Anton Balasingham', 'Asia Tribune vs. Tamilnet'),
 ('Thermal light', 'Reworked'),
 ('Rubber-tyred metro',
  'Rubber Tired Metros as burdensome and having only political (not technical) merit'),
 ('Tour de France', 'Armstrong'),
 ('Hangul', 'Requested move'),
 ('Mount Hebron', 'Removal of well sourced material'),
 ('Croatia national football team', 'Pre independence or 1990s?'),
 ('Satan (South Park)', 'Proposed removal of content'),
 ('Acharya S', 'Delete actual name'),
]

In [24]:
for art, thread in tqdm.tqdm_notebook(threads):
    
    tqdm.tqdm.write(art)
    rows = examples[(examples['article']==art) & (examples['thread']==thread)]
    out = rows.loc[:, ['timestamp', 'username', 'thread_type', 'text', 'editor_score']]

    # Build html_text
    out['html_text'] = ['<div class="thread_text"><br><hr>{:s} <br><br><span class="meta">--{:s} ({:.03f}), {}</span><br></div>'.format(t,u,s,ts) \
                        for t,u,s,ts in zip(out['text'], out['username'], out['editor_score'], out['timestamp'])]
    out.sort_values(['timestamp'], inplace=True)

    # Post-process
    out['html_text'] = [re.sub(r'<div class="thread_text"><br><hr>:+', '<div class="thread_text"><br><hr>', text) for text in out['html_text']]
    
    # Get revision ids before and after discussion
    beg_ts = min(out[out['timestamp'].map(lambda x: isinstance(x, pd.tslib.Timestamp))]['timestamp']).strftime("%Y%m%d%H%M%S")
    end_ts = max(out[out['timestamp'].map(lambda x: isinstance(x, pd.tslib.Timestamp))]['timestamp']).strftime("%Y%m%d%H%M%S")

    page_url = "https://en.wikipedia.org/w/api.php?action=query&titles={:s}&prop=revisions&rvlimit=1&rvstart={:s}&format=json"

    url_specific = {}
    url_specific['beg'] = page_url.format(urllib.parse.quote(art), beg_ts)
    url_specific['end'] = page_url.format(urllib.parse.quote(art), end_ts)
#     print(url_specific)

    revd = {}
    revd['beg'] = json.loads(urllib.request.urlopen(url_specific['beg']).read().decode('utf-8'))
    revd['end'] = json.loads(urllib.request.urlopen(url_specific['end']).read().decode('utf-8'))

    pageid = list(revd['beg']['query']['pages'].keys())[0]
    revid = {}
    revid['beg'] = revd['beg']['query']['pages'][pageid]['revisions'][0]['revid']
    revid['end'] = revd['end']['query']['pages'][pageid]['revisions'][0]['revid']
#     print(revid)

    # Build revision url
    rev_url = {}
    rev_url['beg'] = 'https://en.wikipedia.org/w/index.php?title={}&oldid={}'.format(urllib.parse.quote(art), revid['beg'])
    rev_url['end'] = 'https://en.wikipedia.org/w/index.php?title={}&oldid={}'.format(urllib.parse.quote(art), revid['end'])
    diff_url = 'https://en.wikipedia.org/w/index.php?title={}&diff={}&oldid={}'.format(art, revid['end'], revid['beg'])

    # Add page and thread title
    frame_html = '<iframe src="{}"><p>Your browser does not support iframes.</p></iframe>'.format(rev_url['beg']) + \
                  '<iframe src="{}"><p>Your browser does not support iframes.</p></iframe>'.format(rev_url['end']) + \
                '<iframe class="diff" src="{}"><p>Your browser does not support iframes.</p></iframe>'.format(diff_url)
    hdr = pd.DataFrame([{'html_text': '<span class="page_name">Page: {}</span><br><div class="thread">Thread: {}</div>'.format(art, thread)},
                       {'html_text': frame_html}])
    out = hdr.append(out)
    
    outname = '/home/michael/school/research/wp/jsalt_tutorial/vis/{}.html'.format(art.lower().replace(' ', '_').replace('/', '_'))
    pd.DataFrame(out['html_text']).to_html(outname, escape=False, header=False, border=0, index=False)

German occupation of Czechoslovakia
Anton Balasingham
Thermal light
Rubber-tyred metro
Tour de France
Hangul
Mount Hebron
Croatia national football team
Satan (South Park)
Acharya S



In [19]:
diff_url = 'https://en.wikipedia.org/w/index.php?title={}&diff={}&oldid={}'.format(art, revid['end'], revid['beg'])
diff_url

'https://en.wikipedia.org/w/index.php?title=Acharya S&diff=587878034&oldid=581995654'

In [25]:
# Create wrapper html files
html_content = \
"""
<html>
<head>
<link rel="stylesheet" type="text/css" href="style.css">
<script src="jquery-3.2.1.min.js"></script> 
    <script> 
    $(function(){{
      $("#content").load("{}.html"); 
	$("iframe").css('width', $(window).width()+'px');
	$("iframe").css('height', '600px');
    }});
</script>
<title>{}</title>
  </head> 

  <body> 
     <div id="content"></div>
  </body> 
</html>

"""

page_ctr = 0
for art,_ in threads:
    print(page_ctr, end=': ')
    print(art)
    with open('/home/michael/school/research/wp/jsalt_tutorial/vis/page{}.html'.format(page_ctr), 'w') as f:
        f.write(html_content.format(art.lower().replace(' ', '_').replace('/', ' '), art))
    page_ctr += 1

0: German occupation of Czechoslovakia
1: Anton Balasingham
2: Thermal light
3: Rubber-tyred metro
4: Tour de France
5: Hangul
6: Mount Hebron
7: Croatia national football team
8: Satan (South Park)
9: Acharya S


# Old, for just one thread

In [3]:
# Test thread
art, thread = 'Acharya S', 'Delete actual name'
rows = examples[(examples['article']==art) & (examples['thread']==thread)]
out = rows.loc[:, ['timestamp', 'username', 'thread_type', 'text', 'editor_score']]

# Build html_text
out['html_text'] = ["<br><hr>{:s} <br><br><span class='meta'>--{:s} ({:.03f}), {}</span><br>".format(t,u,s,ts) \
                    for t,u,s,ts in zip(out['text'], out['username'], out['editor_score'], out['timestamp'])]
out.sort_values(['timestamp'], inplace=True)

# Post-process
# out['html_text'] = [re.sub(r'^:+', '', text) for text in out['html_text']]
out['html_text'] = [re.sub(r'<br><hr>:+', '<br><hr>', text) for text in out['html_text']]

## Get beginning, ending revision id from timestamp

In [4]:
beg_ts = min(out[out['timestamp'].map(lambda x: isinstance(x, pd.tslib.Timestamp))]['timestamp']).strftime("%Y%m%d%H%M%S")
end_ts = max(out[out['timestamp'].map(lambda x: isinstance(x, pd.tslib.Timestamp))]['timestamp']).strftime("%Y%m%d%H%M%S")

page_url = "https://en.wikipedia.org/w/api.php?action=query&titles={:s}&prop=revisions&rvlimit=1&rvstart={:s}&format=json"

url_specific = {}
url_specific['beg'] = page_url.format(urllib.parse.quote("Acharya S"), beg_ts)
url_specific['end'] = page_url.format(urllib.parse.quote("Acharya S"), end_ts)
print(url_specific)

revd = {}
revd['beg'] = json.loads(urllib.request.urlopen(url_specific['beg']).read().decode('utf-8'))
revd['end'] = json.loads(urllib.request.urlopen(url_specific['end']).read().decode('utf-8'))

pageid = list(revd['beg']['query']['pages'].keys())[0]
revid = {}
revid['beg'] = revd['beg']['query']['pages'][pageid]['revisions'][0]['revid']
revid['end'] = revd['end']['query']['pages'][pageid]['revisions'][0]['revid']
print(revid)

# Build revision url
rev_url = {}
rev_url['beg'] = 'https://en.wikipedia.org/w/index.php?title={}&oldid={}'.format(urllib.parse.quote("Acharya S"), revid['beg'])
rev_url['end'] = 'https://en.wikipedia.org/w/index.php?title={}&oldid={}'.format(urllib.parse.quote("Acharya S"), revid['end'])
print(rev_url['beg'])
print(rev_url['end'])

{'beg': 'https://en.wikipedia.org/w/api.php?action=query&titles=Acharya%20S&prop=revisions&rvlimit=1&rvstart=20131119222403&format=json', 'end': 'https://en.wikipedia.org/w/api.php?action=query&titles=Acharya%20S&prop=revisions&rvlimit=1&rvstart=20131228112503&format=json'}
{'beg': 581995654, 'end': 587878034}
https://en.wikipedia.org/w/index.php?title=Acharya%20S&oldid=581995654
https://en.wikipedia.org/w/index.php?title=Acharya%20S&oldid=587878034


## Header with info and URLs

In [5]:
# Add page and thread title
frame_html = '<iframe src="{}"><p>Your browser does not support iframes.</p></iframe>'.format(rev_url['beg']) + \
              '<iframe src="{}"><p>Your browser does not support iframes.</p></iframe>'.format(rev_url['end'])
hdr = pd.DataFrame([{'html_text': '<span class="page_name">Page: {}</span><br><div class="thread">Thread: {}</div>'.format(art, thread)},
                   {'html_text': frame_html}])
out = hdr.append(out)
out

,editor_score,html_text,text,thread_type,timestamp,username
0,NaN,"<div class=""page_name"">Page: Acharya S</div><b...",NaN,NaN,NaT,NaN
1,NaN,"<iframe src=""https://en.wikipedia.org/w/index....",NaN,NaN,NaT,NaN
10,0.416667,"<br><hr>Her ""stalkers"" must be unaware of this...",":Her ""stalkers"" must be unaware of this thing ...",POST,2013-11-19 22:24:03,Capitalismojo
8,0.706074,<br><hr>Acharya S responded on her Facebook pa...,Acharya S responded on her Facebook page that ...,THREAD_STARTER,2013-11-20 04:52:07,RL0919
9,0.706074,"<br><hr>To clarify, what she claims on Faceboo...",":::To clarify, what she claims on Facebook is ...",POST,2013-11-20 04:52:07,RL0919
12,0.659794,<br><hr>someone who is out there publishing an...,::someone who is out there publishing and prom...,POST,2013-12-13 22:09:46,TheRedPenOfDoom
14,0.309613,<br><hr>Were the two editors directly above in...,:::Were the two editors directly above invited...,POST,2013-12-14 02:25:28,In ictu oculi
15,0.309613,"<br><hr>TEMPLATE[ping, In ictu oculi] Sorry, I...","::::TEMPLATE[ping, In ictu oculi] Sorry, I did...",POST,2013-12-25 18:57:04,In ictu oculi
19,0.500000,<br><hr>How are the first two reliable sources...,:::::::How are the first two reliable sources ...,POST,2013-12-25 19:04:49,John Reaves
20,0.500000,<br><hr>How is any book giving anyone's name r...,::::::::How is any book giving anyone's name r...,POST,2013-12-25 20:21:07,John Reaves


In [6]:
pd.set_option('display.max_colwidth', -1)

In [7]:
pd.DataFrame(out['html_text']).to_html('/home/michael/school/research/wp/jsalt_tutorial/vis/test_vis.html', escape=False, header=False, border=0, index=False)

# Find 10 interesting conversations

In [20]:
talk = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/talk_filtered_3.csv')
talk

,contribution_id,article,thread,timestamp,username,thread_type,text
0,1,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,THREAD_STARTER,"The following discussion is an archived discussion of a requested move. Please do not modify it. Subsequent comments should be made in a new section on the talk page. No further edits should be made to this section.\nThe result of the move request was: move lapsed, article merge preferred billinghurst sDrewth 23:13, 21 July 2010 (UTC)"
1,2,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)"
2,3,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop
3,4,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)"
4,5,"""Tiger"" William Dunlop",Requested move,2010-07-14 15:12:03,Skinsmoke,POST,"Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. Skinsmoke (talk) 21:12, 14 July 2010 (UTC)"
5,6,"""Tiger"" William Dunlop",Requested move,2010-07-14 16:59:04,Deconstructhis,POST,"Not a problem to merge if that is the most appropriate way to accomplish a decent article on Tiger. I am new to wikiland so bear with me and try not to use the royal ""we"" with me--it's a little high handed; there is more neutral way to say that. —Preceding unsigned comment added by CJ3370 (talk • contribs) 22:38, 14 July 2010 (UTC)\nExcellent job! Dunlop has been deserving of a much more informative presentation of both his interesting life and its impact on the development of southwestern Ontario in proper article form for years now. Thank you for your time and effort. Welcome to Wikipedia! Cheers Deconstructhis (talk) 22:59, 14 July 2010 (UTC)"
6,7,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,"Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! Skinsmoke (talk) 01:49, 15 July 2010 (UTC)"
7,8,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,POST,The above discussion is preserved as an archive of a requested move. Please do not modify it. Subsequent comments should be made in a new section on this talk page. No further edits should be made to this section.
8,9,"""Tiger"" William Dunlop",Current image,2010-07-15 20:19:17,Deconstructhis,THREAD_STARTER,"I've noticed that the current image that's accompanying the article is marked for deletion on 21 July because of licensing issues. That's too bad really, I always think that any half decent picture at the top of the article really gives the initial presentation an added boost. Trust me, I'm the last person that should directly advise another editor on how to clear up these kinds of image 'problems'; but I know we have lots of editors around here that are quite expert at providing advice on contingencies. If I have some time over the next several days, I'll poke around and perhaps try to locate a good venue in the form of a noticeboard to seek some answers. I wonder how difficult it woul

In [21]:
thread_lens = talk.groupby(['article', 'thread']).size()
thread_lens

article                        thread                                                  
"Tiger" William Dunlop         Current image                                               5 
                               Requested move                                              8 
"V" Is for Vagina              Why is the "i" capitalized?                                 10
&                              "O" as a word                                               4 
                               And per se and as a letter                                  5 
                               My use                                                      6 
                               Programming languages                                       4 
                               Usage                                                       3 
                               Wikipedia:Manual of Style (And vs. &)                       5 
'Abd al-Ilah                   Arabic spelling                    

In [22]:
thresholds = [5, 10, 15, 20, 30, 40]
binned_threads = {min_len: list() for min_len in thresholds}

for min_len in thresholds:
    binned_threads[min_len] = thread_lens[thread_lens > min_len].index.tolist()

binned_threads

{5: [('"Tiger" William Dunlop', 'Requested move'),
  ('"V" Is for Vagina', 'Why is the "i" capitalized?'),
  ('&', 'My use'),
  ("(I Can't Get No) Satisfaction", 'Britney Spears cover'),
  ('.hack//Outbreak', 'Lead image'),
  ('.hack//Outbreak', 'Removed text'),
  ('.tib files', 'Merger proposal'),
  ('11:00PM-12:00AM (24 season 6)',
   'DO NOT EDIT THIS PAGE WITHOUT HARD FACTS'),
  ('11:00PM-12:00AM (24 season 6)', 'Date Error'),
  ('11:00PM-12:00AM (24 season 6)', 'Death Count'),
  ('11:00PM-12:00AM (24 season 6)', 'The episodes have not aired yet on TV.'),
  ('1587 Yellow River flood', 'Length'),
  ('1587 Yellow River flood', 'Mongolian name'),
  ('17-Mile Drive', '17 Mile Drive info box and section images replacement'),
  ('17-Mile Drive', 'RFC on Lead Image'),
  ('17-Mile Drive', 'Set C'),
  ('180 Solutions', '2014 Neutrality discussion'),
  ('180 Solutions', 'An update on the spyware status?'),
  ('180 Solutions', 'Private Message'),
  ('180 Solutions', "Yes, it's spyware"),
  ('

In [6]:
for min_len in thresholds:
    print(min_len, end=': ')
    print(len(binned_threads[min_len]))

10: 2164
15: 1090
20: 617
30: 220
40: 101


In [8]:
# Sample from 15-threshold threads
sample = random.sample(binned_threads[15], 10)
sample

[('Anton Balasingham', 'Asia Tribune vs. Tamilnet'),
 ('Oh Eun-sun', 'Biased Edits'),
 ('Yoni Jesner', 'DYK preparation'),
 ('Greater Assam', 'Other states of India:- Citation needed.'),
 ('Japanese acupuncture', 'Original research in the article again'),
 ('MAO College',
  'Undue weight, duplicative coverage and violation of Summary Style'),
 ('Commodity fetishism',
  'when words are scarce they are seldom spent in vein'),
 ('Belgic people', 'Geography'),
 ('Einstein', 'Including relevant quotes'),
 ('E-101 Beta',
  'RfC: Do the characters discussed here meet WP:N such that they should be allowed to keep their articles?')]

In [14]:
# Sample from 10-threshold threads
samples = {}
samples[10] = random.sample(binned_threads[10], 10)
samples[10]

[('Polish occupation of Czechoslovakia',
  'Extent of coverage of the postwar expulsions'),
 ('Priesthood keys', 'Priesthood (Mormonism) Chart'),
 ('US Airways Marketplace', 'Twitter Pornographic Incident'),
 ('Acharya S', 'Delete actual name'),
 ('Eric Harris and Dylan Klebold', 'Marilyn Manson/Misleading use of quote'),
 ('Sandra Caron', '"Non-free" image use'),
 ('Satish Dhawan Space Centre', 'Why 4 articles?'),
 ('Neutral wire', 'Is  Neutral a Current-Carrying Conductor?'),
 ('Kiev Pechersk Lavra', 'English name with "caves"'),
 ('Late Imperial China', 'Modern history')]

In [23]:
# Sample from 5-threshold threads
t = 5
samples = {}
samples[t] = random.sample(binned_threads[t], 10)
samples[t]

[('2006 World Series of Poker results', 'main event results'),
 ('Custom House for ExCeL DLR station', 'Requested move'),
 ('Ukrajina',
  "Request for comment: number of subdivisions in lead: display or don't"),
 ('Lunitari', 'Merger with individual deities'),
 ('Journal box', 'Add Through routed defintion'),
 ('Hangul', 'Requested move'),
 ('Croatia national football team', 'Pre independence or 1990s?'),
 ('Episode 3.10', 'Webisode numbering'),
 ('John Forbes Nash, Jr.', "References for Nash's sexuality"),
 ('Broadhorn', 'UKW Relevence')]

In [24]:
pd.set_option('display.max_colwidth', 999)

In [32]:
n = 6
a,t = samples[5][n]
rows = talk[(talk['article']==a) & (talk['thread']==t)]
rows

# for a,t in sample:
#     rows = talk[(talk['article']==a) & (talk['thread']==t)]
#     print(rows)

,contribution_id,article,thread,timestamp,username,thread_type,text
22200,49227,Croatia national football team,Pre independence or 1990s?,2008-08-21 01:35:06,Domiy,THREAD_STARTER,"I think this following statement should go under the 'Pre independence' subheading:\n---The last Yugoslav team to field a significant Croatian contingent played against Sweden on May 16, 1991, just a few days before Croatia's independence referendum.[14] The modern Croatian team played their first international in a friendly game against the United States on October 17, 1990 at Maksimir Stadium. Aljoša Asanović counted as the first goalscorer for the newly established side, as Croatia won the match 2–1.[15] When the match took place, Croatia was still regarded as part of Yugoslavia.[16] ---\nUp to that point, it is clearly stated that Croatia did not yet become independent. It even says that they were still a part of Yugoslavia, meaning it is clearly actions that took place before independence, hence it really should go under the first subheading. Unless of course we change from 'Pre independence' to something like '1940s' or another clever heading which implies earlier/debuting d..."
22201,49228,Croatia national football team,Pre independence or 1990s?,2008-08-21 13:09:46,Timbouctou,POST,"I think it should be included in the 1990s section, since the article is mainly concerned with the history of Croatia football team, not Croatia the country. Since the USA game is regarded as the first game in the modern history of the Croatian national team, it shouldn't really matter what was the legal status of Croatia at the time. Besides, their first games were only months before Croatia officially decalred idependence and stats from these matches are normally included in official UEFA goalscoring and app statistics for Croatia's national football team. Timbouctou (talk) 19:09, 21 August 2008 (UTC)"
22202,49229,Croatia national football team,Pre independence or 1990s?,2008-08-21 16:05:51,Domiy,POST,"Changed it back to previous via fair point. Domiy (talk) 22:05, 21 August 2008 (UTC)"
22203,49230,Croatia national football team,Pre independence or 1990s?,2008-08-21 16:40:03,Domiy,POST,"Comment While we are on such issues, can I ask who created the diagram of the 3-5-2 lineup during the 98 World Cup? Something thats been bothering me for so long now, it is actually backwards! Looking at it from the right position actually shows it as a 2-5-3 formation. It should go the other way! The defenders should be at the bottom and the strikers should be at the top! I dont know how this thing works, can anyone please fix it? Domiy (talk) 22:40, 21 August 2008 (UTC)"
22204,49231,Croatia national football team,Pre independence or 1990s?,2008-09-04 20:48:54,N1cky,POST,"I created the ""diagram"" ages ago. Although i disagree i'll fix it up for ya tomorrow. 2day i don't have too much time. BTW, this is featured on few more articles and it looks the same. Bayern Munich article for instance... It's the way you look at it i guess... Kind regards. Malez"
22205,49232,Croatia national football team,Pre independence or 1990s?,2008-09-04 20:51:36,Domiy,POST,"THANKS! I just think it makes it look confusing. It is much more easier to read when it is the other way like I said before. No rush, just reverse it when you have time. This failed FA again because of some unfair racial issues above all, so I've taken this to Good Article nomination and we'll go from there. I'm just trying to find ways to fix/improve this article further. Above that, trying to replace Javno.com as a reference even though they are fairly reliable in Europe and here in Australia. Ajde onda cujemo se. Zbogom! Domiy (talk) 02:51, 5 September 2008 (UTC)"


# Gather samples into CSV

In [71]:
selected = [
    ('Anton Balasingham', 'Asia Tribune vs. Tamilnet'),
    ('German occupation of Czechoslovakia', 'Extent of coverage of the postwar expulsions'),
    ('Acharya S', 'Delete actual name'),
    ('Rubber tired metro', 'Rubber Tired Metros as burdensome and having only political (not technical) merit'),
    ('Thermal light', 'Reworked'),
    ('Satan (South Park)', 'Proposed removal of content'),
    ('Mount Hebron', 'Removal of well sourced material'),
    ('Tour de France', 'Armstrong'),
    ('Hangul', 'Requested move'),
    ('Croatia national football team', 'Pre independence or 1990s?')
]
len(selected)

10

In [48]:
# Put in editor scores
scores = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores.csv')
scores.columns

Index(['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score'],
      dtype='object')

In [49]:
merged = pd.merge(talk, scores, left_on=['article', 'thread', 'username'], right_on=['article', 'thread_title', 'editor'])
merged = merged[talk.columns.tolist() + ['editor_score']]
merged

,contribution_id,article,thread,timestamp,username,thread_type,text,editor_score
0,2,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)",0.826415
1,2,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)",0.826415
2,4,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)",0.826415
3,4,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)",0.826415
4,3,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop,0.456897
5,3,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop,0.456897
6,5,"""Tiger"" William Dunlop",Requested move,2010-07-14 15:12:03,Skinsmoke,POST,"Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. Skinsmoke (talk) 21:12, 14 July 2010 (UTC)",0.456897
7,5,"""Tiger"" William Dunlop",Requested move,2010-07-14 15:12:03,Skinsmoke,POST,"Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. Skinsmoke (talk) 21:12, 14 July 2010 (UTC)",0.456897
8,7,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,"Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! Skinsmoke (talk) 01:49, 15 July 2010 (UTC)",0.456897
9,7,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,"Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! Skinsmoke (talk) 01:49, 15 July 2010 (UTC)",0.456897


In [50]:
cols = merged.columns.tolist()
new_cols = cols[:3] + ['username', 'timestamp'] + cols[5:]
merged = merged[new_cols]

In [51]:
mask = list(map(lambda tup: tup in selected, zip(merged['article'], merged['thread'])))
selected_rows = merged[mask]
selected_rows

,contribution_id,article,thread,username,timestamp,thread_type,text,editor_score
2027,6395,Acharya S,Delete actual name,RL0919,2013-11-20 04:52:07,THREAD_STARTER,"Acharya S responded on her Facebook page that due to her occupation she feels duress from stalkers knowing her ""actual name."" Per WP:BLPNAME: When the name of a private individual has not been widely disseminated or has been intentionally concealed, such as in certain court cases or occupations, it is often preferable to omit it, especially when doing so does not result in a significant loss of context. I ask that we remove the name. Raquel Baranow (talk) 16:29, 19 November 2013 (UTC)",0.706074
2028,6397,Acharya S,Delete actual name,RL0919,2013-11-20 04:52:07,POST,":::To clarify, what she claims on Facebook is that the name given is not correct, but people who think it is her name use it in an attempt to intimidate her because they know she has been stalked in the past. The factual denial seems more relevant than the stalking concern. --RL0919 (talk) 10:52, 20 November 2013 (UTC)",0.706074
2029,6396,Acharya S,Delete actual name,Capitalismojo,2013-11-19 22:24:03,POST,":Her ""stalkers"" must be unaware of this thing called ""google"". Capitalismojo (talk) 04:24, 20 November 2013 (UTC)\n::And books. Capitalismojo (talk) 04:26, 20 November 2013 (UTC)",0.416667
2030,6412,Acharya S,Delete actual name,Capitalismojo,2013-12-26 18:46:15,POST,":::::::::::This has been resolved by the recent RfC (see above). If you would like to start another RfC again so soon hoping to get another result, feel free but know that this issue has been resolved. Capitalismojo (talk) 00:46, 27 December 2013 (UTC)",0.416667
2031,6399,Acharya S,Delete actual name,TheRedPenOfDoom,2013-12-13 22:09:46,POST,"::someone who is out there publishing and promoting books is not a ""private individual."" -- TRPoD aka The Red Pen of Doom 04:09, 14 December 2013 (UTC)",0.659794
2032,6410,Acharya S,Delete actual name,TheRedPenOfDoom,2013-12-28 11:25:03,POST,"::::::::::::::::Madonna Louise Ciccone is not her preference and appears in about the same proportion. Murdock is not a ""private"" individual, she is a public figure who has promoted her work. -- TRPoD aka The Red Pen of Doom 17:25, 28 December 2013 (UTC)",0.659794
2033,6400,Acharya S,Delete actual name,In ictu oculi,2013-12-14 02:25:28,POST,":::Were the two editors directly above invited here by email? I think this was considered in the RfC and the 4 book sources all use the name ""Dorothy M. Murdock"" and two use ""Dorothy M___ Murdock"" while referencing it to the author's own website so this seems to be rather ridiculous. WP:FULLNAME applies to leads in all biographies including ""Chelsea Elizabeth Manning[4] (born Bradley Edward Manning, December 17, 1987) so I'm not sure what the comparison proves. If anything we shouldn't really have M. when Holding and the school source the middle name. In ictu oculi (talk) 08:25, 14 December 2013 (UTC)",0.309613
2034,6401,Acharya S,Delete actual name,In ictu oculi,2013-12-25 18:57:04,POST,"::::TEMPLATE[ping, In ictu oculi] Sorry, I did not realize there had been an RfC. Full disclosure: I have interacted with the subject via OTRS. I changed to the initials because the subject has actually published under them unlike the full name. Are there instances of her having actually published under the name ""Dorothy"" or admitting that is here real name? Annual Report - American School of Classical Studies at Athens doesn't show any proof and the other two sources surely can't be said to be reliable for her full name. -- John Reaves 00:52, 26 December 2013 (UTC)\n::::::The two additional book print sources, one ""Dorothy M. Murdock"", on ""Dorothy Milne Murdock"" are reliable sources. The summer school Annual Report - American School of Classical Studies at Athens was introduced by the author herself on her own website as part of author's expertise/credentials. In ictu oculi (talk) 00:57, 26 December 2013 (UTC)",0.309613
20

In [52]:
selected_rows.to_csv('/home/michael/school/research/wp/jsalt_tutorial/examples.csv', index=False)

In [74]:
example_dfs = [merged[(merged['article']==a) & (merged['thread']==t)] for a,t in selected]
len(example_dfs)

10

In [75]:
writer = ExcelWriter('/home/michael/school/research/wp/jsalt_tutorial/wikipedia_talk_page_examples.xlsx')

for df in example_dfs:
    df.to_excel(writer, df['article'].unique().tolist()[0][:31], index=False)
    
writer.save()

In [76]:
[df['article'].unique() for df in example_dfs]

[array(['Anton Balasingham'], dtype=object),
 array(['German occupation of Czechoslovakia'], dtype=object),
 array(['Acharya S'], dtype=object),
 array(['Rubber tired metro'], dtype=object),
 array(['Thermal light'], dtype=object),
 array(['Satan (South Park)'], dtype=object),
 array(['Mount Hebron'], dtype=object),
 array(['Tour de France'], dtype=object),
 array(['Hangul'], dtype=object),
 array(['Croatia national football team'], dtype=object)]

In [65]:
# Attempt at Polish to German (name replacement)
merged.replace(to_replace={'article': {'Polish occupation of Czechoslovakia': 'German occupation of Czechoslovakia'}}, inplace=True)